# Conversational ChatBot using Deeppavlov SQUAD BERT Transformer

### Install required libraries

In [ ]:
!pip install deeppavlov
# !pip install transformers

In [ ]:
!python -m deeppavlov install squad_bert

2021-09-29 06:04:46.558 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-jqijx3yo
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-jqijx3yo


## Import required libraries

In [ ]:
from os import path
import json 
import pickle

### for deep neural network - intent classification
import numpy as np 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

### deeppavlov model - for question-answering
from deeppavlov import build_model, configs

### Set directory path 
dir_path = r"/content/"

### initialize all required variables as a global variables
model = None 
model_squad = None 
tokenizer = None 
label_encoder = None 

### load the intend file in a global scope
with open(dir_path + r"intents_medical.json") as intent_file:
    data = json.load(intent_file)

# Function for building and loading model 

In [ ]:
def define_models():

    ''' the function will define/initialize all the required models and save them for future use '''

    # access global variable to modify them
    global model, model_squad, tokenizer, label_encoder #, tokenizer_bert

    if path.exists(dir_path + r"model_chatbot.h5"): 

        # load the saved model for intent classification using keras
        model = keras.models.load_model(dir_path + r"model_chatbot.h5")

        # load the deeppavlov squad model for question-answering 
        model_squad = build_model(configs.squad.squad, download=True)

        # load the bert squad model for question-answering 
        # model_squad = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

        # load the tokenizer_bert for question-answering
        # tokenizer_bert = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

        # load the saved tokenizer object using pickle
        with open(dir_path + r"tokenizer.pickle", mode = 'rb') as token_load:
            tokenizer = pickle.load(token_load)

        # load the saved label encoder object using pickle
        with open(dir_path + r'label_encoder.pickle', mode = 'rb') as label_load:
            label_encoder = pickle.load(label_load)

        print("Model is loaded...")

    else:

        training_sentences = []
        training_labels = []
        labels = []
        
        for intent in data['intents']:
            for pattern in intent['text']:
                training_sentences.append(pattern)
                training_labels.append(intent['intent'])
            
            if intent['intent'] not in labels:
                labels.append(intent['intent'])
                
        num_classes = len(labels)
        
        label_encoder = LabelEncoder()
        label_encoder.fit(training_labels)
        training_labels = label_encoder.transform(training_labels)
        
        vocab_size = 1000
        embedding_dim = 16
        max_len = 20
        oov_token = "<OOV>"
        
        tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
        tokenizer.fit_on_texts(training_sentences)
        word_index = tokenizer.word_index
        sequences = tokenizer.texts_to_sequences(training_sentences)
        padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
        
        model = Sequential()
        model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(16, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        print(model.summary())
        
        epochs = 500
        model.fit(padded_sequences, np.array(training_labels), epochs=epochs)
        
        # save the trained model using keras
        model.save(dir_path + r"model_chatbot.h5")

        # to save the fitted tokenizer
        with open(dir_path + r'tokenizer.pickle', mode = 'wb') as token_save:
            pickle.dump(tokenizer, token_save, protocol=pickle.HIGHEST_PROTOCOL)
            
        # to save the fitted label encoder
        with open(dir_path + r'label_encoder.pickle', mode = 'wb') as label_save:
            pickle.dump(label_encoder, label_save, protocol=pickle.HIGHEST_PROTOCOL)

        # load the deeppavlov squad model for question-answering 
        model_squad = build_model(configs.squad.squad, download=True)

        # load the bert squad model for question-answering 
        # model_squad = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

        # load the tokenizer_bert for question-answering
        # tokenizer_bert = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

        print("Model is loaded...")

# Chat Function

In [ ]:
def chat(que):

    ''' firstly, the function will predict the intent and then find the answer of the quetion asked by the user  '''

    # parameters
    max_len = 20
    
    while True:
        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([que]), truncating='post', maxlen=max_len))
        tag = label_encoder.inverse_transform([np.argmax(result)])

        answer = ""

        for i in data['intents']:
            if i['intent'] == tag:

                res = np.random.choice(i['responses'])

                # Approach-1: question-answering Using deeppavlov squad model
                out = model_squad([res], [que])
                return answer.join(out[0])

# Main Function

In [ ]:
def main():  
  ''' the function where human and chatbot interaction is happening '''
  
  ### Ask your question related to home remedies..
  print("Hi, I'm Lara. Ask your medical questions here. (type bye to stop)...")
  while True:  
    # taking user input
    user_input = input('Me: ').lower()
    if user_input == "bye":
      print("Lara: Bye, take care...")
      break
      
    else:
      # text generation by bot
      bot_output = chat(user_input)
      print("Lara: ", bot_output)

# Building and loading the models

In [ ]:
define_models()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 45)                765       
Total params: 17,309
Trainable params: 17,309
Non-train

2021-09-29 06:10:14.65 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M-char.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M-char.vec
100%|██████████| 7.73M/7.73M [00:03<00:00, 2.20MB/s]
2021-09-29 06:10:22.563 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_1.4_cpu_compatible.tar.gz to /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.gz
100%|██████████| 222M/222M [01:42<00:00, 2.16MB/s]
2021-09-29 06:12:07.381 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.gz archive into /root/.deeppavlov/models
2021-09-29 06:12:14.368 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M.vec
100%|█████████

2021-09-29 06:33:30.146 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead 

2021-09-29 06:33:31.238 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
2021-09-29 06:33:31.412 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
2021-09-29 06:33:31.518 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-09-29 06:33:45.635 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_model/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_model/model
Model is loaded...


# Let's Chat....

In [ ]:
main()

Hi, I'm Lara. Ask your medical questions here. (type bye to stop)...
Me: how to cure cough?
Lara:  Use honey
Me: how to treat mild fever
Lara:  Drink plenty of fluids to stay hydrated
Me: what to apply on cuts?
Lara:  pressure for 1-2minutes
Me: how to cure fainting?
Lara:  Lie down or sit down
Me: how to stop bleeding?
Lara:  direct pressure
Me: which medicine to take if i get a mild fever
Lara:  Take acetaminophen
Me: how do you treat heat exhaustion?
Lara:  Remove tight or heavy clothing
Me: bye
Lara: Bye, take care...


In [ ]:
main()

Hi, I'm Lara. Ask your medical questions here. (type bye to stop)...
Me: i'm having headache please help me
Lara:  to help with the discomfort and swelling of sunburn
Me: fever
Lara:  Drink plenty of fluids to stay hydrated
Me: mild headache
Lara:  ibuprofen (Advil, Motrin), aspirin, or acetaminophen (Tylenol) for pain
Me: bye
Lara: Bye, take care...
